# Applied Data Science Capstone
## Week 4 & 5

### Introduction

This project looks at the relationships between residentail project values and surrounding venues (e.g., venue categories, ratings). It is likely that new residential projects are constructed with the consideration of surrounding facilities. Thus larger residential project investments may correlate with better and more diversed neighbouring venues. 

### Data

This project will restrict to 2019 new residential buildings in the city of Baton Rouge, Louisiana. Residentail project value data is found at https://data.brla.gov/Housing-and-Development/New-Residential-Building-Permits-by-Month/2vpm-jgm6. The data source provides information including buildings' geolocations, addresses, and projects' value. 